# Meeting Prep and Strategy Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/meeting_prep_and_strategy_agents.ipynb)

## Dependencies

In [69]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install duckduckgo-search > /dev/null
%pip install exa_py > /dev/null

## Tools

In [77]:
#Replaced Exa Search tool with InternetSearchTool for simplicity
import os
import requests
from bs4 import BeautifulSoup
from praisonai_tools import BaseTool
from langchain.tools import tool
from exa_py import Exa

from duckduckgo_search import DDGS


class ExaSearchTool(BaseTool):
    name: str = "ExaSearchTool"
    description: str = "A tool to search for and retrieve webpages or similar content based on a query or URL."

    def _run(self, query: str) -> str:
        """Default method for running a simple search."""
        return self.search(query)

    @classmethod
    def tools(cls):
        """Returns all tool functions as a list."""
        return [search_tool, find_similar_tool, get_contents_tool]


@tool("Search Webpages")
def search_tool(query: str) -> str:
    """Searches for webpages based on the query string and returns up to 3 results."""
    exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
    return exa_instance.search(query, use_autoprompt=True, num_results=3)


@tool("Find Similar Webpages")
def find_similar_tool(url: str) -> str:
    """Finds webpages similar to a specified URL and returns up to 3 results."""
    exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
    return exa_instance.find_similar(url, num_results=3)


@tool("Get Webpage Contents")
def get_contents_tool(ids: str) -> str:
    """Retrieves the contents of specified webpage IDs (as a list) and returns up to 1000 characters per page."""
    exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
    ids_list = eval(ids)  # Convert string input to a list
    contents = exa_instance.get_contents(ids_list)
    formatted_contents = [
        content[:1000] for content in str(contents).split("URL:")  # Limit each content to 1000 characters
    ]
    return "\n\n".join(formatted_contents)


class InternetSearchTool(BaseTool):
    name: str = "InternetSearchTool"
    description: str = "Search Internet for relevant information based on a query or latest news"

    def _run(self, query: str):
        ddgs = DDGS()
        results = ddgs.text(keywords=query, region='wt-wt', safesearch='moderate', max_results=5)
        return results


## YAML Prompt

In [80]:
agent_yaml = """
framework: "crewai"
topic: "Meeting Preparation and Strategy Development"
roles:
  research_specialist:
    role: "Research Specialist"
    backstory: |
      As a Research Specialist, you are dedicated to uncovering detailed information
      about the individuals and entities involved in the meeting. Your insights lay
      the groundwork for strategic preparation and effective engagement.
    goal: "Conduct thorough research on meeting participants and organizations."
    tasks:
      research_task:
        description: |
          Conduct comprehensive research on each of the individuals and companies involved in the upcoming meeting.
          Gather information on recent news, achievements, professional background, and relevant business activities.
          Participants: ["John Doe - CEO, Tech Innovations Inc.", "Jane Smith - Head of AI Research, NextGen AI Labs", "Emily Carter - Chief Strategist, FutureTech Partners"]
          Meeting Context: "Discussing recent AI advancements, strategic partnerships, and potential collaborative projects in the tech industry."
        expected_output: |
          A detailed report summarizing key findings about each participant and company,
          highlighting information that could be relevant for the meeting.
    tools:
      - "InternetSearchTool"
  industry_analyst:
    role: "Industry Analyst"
    backstory: |
      As an Industry Analyst, your focus is on identifying key industry trends, challenges, and opportunities.
      Your analysis provides a comprehensive industry overview that can be leveraged strategically in the meeting.
    goal: "Analyze the current industry trends, challenges, and opportunities."
    tasks:
      industry_analysis_task:
        description: |
          Analyze current industry trends, challenges, and opportunities relevant to the meeting's context.
          Review market reports, recent developments, and expert opinions to provide a comprehensive industry landscape.
          Participants: ["John Doe - CEO, Tech Innovations Inc.", "Jane Smith - Head of AI Research, NextGen AI Labs", "Emily Carter - Chief Strategist, FutureTech Partners"]
          Meeting Context: "Discussing recent AI advancements, strategic partnerships, and potential collaborative projects in the tech industry."
        expected_output: |
          An insightful analysis that identifies major trends, potential challenges, and strategic opportunities.
    tools:
      - "InternetSearchTool"
  meeting_strategy_advisor:
    role: "Meeting Strategy Advisor"
    backstory: |
      As a Strategy Advisor, you specialize in developing talking points, questions, and discussion angles to ensure
      the meeting's objectives are achieved. Your expertise supports the meeting’s strategic direction.
    goal: "Develop talking points, questions, and strategic angles for the meeting."
    tasks:
      meeting_strategy_task:
        description: |
          Develop strategic talking points, questions, and discussion angles for the meeting based on the research and industry analysis conducted.
          Meeting Context: "Discussing recent AI advancements, strategic partnerships, and potential collaborative projects in the tech industry."
          Meeting Objective: "Establishing partnership opportunities, aligning goals in AI research, and identifying key collaboration areas to strengthen market positioning."
        expected_output: |
          Complete report with a list of key talking points, strategic questions to ask, and angles to approach to help achieve the meeting’s objectives.
    tools:
      - "InternetSearchTool"
  briefing_coordinator:
    role: "Briefing Coordinator"
    backstory: |
      As the Briefing Coordinator, your role is to consolidate all research findings, analysis, and strategic insights.
      You provide a structured, easy-to-digest briefing that equips participants with necessary information and strategies.
    goal: "Compile all gathered information into a concise, informative briefing document."
    tasks:
      summary_and_briefing_task:
        description: |
          Compile all research findings, industry analysis, and strategic talking points into a concise, comprehensive briefing document.
          Ensure the briefing is structured and accessible to equip meeting participants with all necessary information and strategies.
          Meeting Context: "Discussing recent AI advancements, strategic partnerships, and potential collaborative projects in the tech industry."
          Meeting Objective: "Establishing partnership opportunities, aligning goals in AI research, and identifying key collaboration areas to strengthen market positioning."
        expected_output: |
          A well-structured briefing document that includes sections for
          participant bios, industry overview, talking points, and strategic recommendations.
    tools:
      - "InternetSearchTool"
dependencies: []
"""

## Main

In [81]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[InternetSearchTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["EXA_API_KEY"] = userdata.get('EXA_API_KEY') or "ENTER EXA_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 10/10


 [2024-11-04 07:29:12][DEBUG]: == Working Agent: Research Specialist
 [2024-11-04 07:29:12][INFO]: == Starting Task: Conduct comprehensive research on each of the individuals and companies involved in the upcoming meeting.
Gather information on recent news, achievements, professional background, and relevant business activities.
Participants: ["John Doe - CEO, Tech Innovations Inc.", "Jane Smith - Head of AI Research, NextGen AI Labs", "Emily Carter - Chief Strategist, FutureTech Partners"]
Meeting Context: "Discussing recent AI advancements, strategic partnerships, and potential collaborative projects in the tech industry."



> Entering new CrewAgentExecutor chain...
I need to gather detailed information about each participant and their respective organizations to prepare for the upcoming meeting. This includes recent news, achievements, professional backgrounds, and relevant business activities. 

Action: InternetSearchTool  
Action Input: {"query": "John Doe CEO Tech Innovations In

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'John Doe - Chief Executive Officer - Google - LinkedIn', 'href': 'https://www.linkedin.com/in/john-doe-1229882ba', 'body': "Welcome back. Chief Executive Officer at Google · Experience: Google · Location: Greater Philadelphia. View John Doe's profile on LinkedIn, a professional community of 1 billion members."}, {'title': 'The 16 Most Influential (And Inspirational) Modern Tech Leaders ...', 'href': 'https://www.forbes.com/councils/forbestechcouncil/2020/07/10/the-16-most-influential-and-inspirational-modern-tech-leaders-according-to-industry-experts/', 'body': '5. Steve Jobs. I believe the most influential tech leader of our era was Steve Jobs. He was so forward-thinking and innovative and utilized out-of-the-box business and product strategies. He ...'}, {'title': 'Case Study: Transforming Company Culture at Tech Innovations Inc.', 'href': 'https://www.linkedin.com/pulse/case-study-transforming-company-culture-mkiqc', 'body': 'Tech Innovations Inc., a renowned software

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'John Doe - Chief Executive Officer - Google - LinkedIn', 'href': 'https://www.linkedin.com/in/john-doe-1229882ba', 'body': "Welcome back. Chief Executive Officer at Google · Experience: Google · Location: Greater Philadelphia. View John Doe's profile on LinkedIn, a professional community of 1 billion members."}, {'title': 'PharmaNova CEO John Doe Reflects on Leadership and Challenges - openPR.com', 'href': 'https://www.openpr.com/news/3609283/pharmanova-ceo-john-doe-reflects-on-leadership-and-challenges', 'body': "Newark, NJ - July 12, 2022 - In an exclusive interview with Newark News Network, John Doe, CEO of PharmaNova Inc., opened up about his tenure leading one of the world's foremost pharmaceutical ..."}, {'title': 'PRESS RELEASE - CEO: John Doe of PharmaNova, Inc. - openPR.com', 'href': 'https://www.openpr.com/news/3609277/press-release-ceo-john-doe-of-pharmanova-inc', 'body': "Newark, NJ - July 12, 2022 - In an exclusive interview with Newark News Network, John Do

...

 

[{'title': 'Four trends that changed AI in 2023 | MIT Technology Review', 'href': 'https://www.technologyreview.com/2023/12/19/1085696/four-trends-that-changed-ai-in-2023/', 'body': "That gives me a lot of hope for the future of AI. Here's what 2023 taught me: 1. Generative AI left the lab with a vengeance, but it's not clear where it will go next. The year started with ..."}, {'title': 'AI in 2023: A year of breakthroughs that left no human thing ... - ZDNET', 'href': 'https://www.zdnet.com/article/ai-in-2023-a-year-of-breakthroughs-that-left-no-human-thing-unchanged/', 'body': 'The $699 Humane AI Pin marked a significant development in wearable technology, blending fashion and tech with its 13MP ultrawide camera and laser projection features. This device highlights the ...'}, {'title': '13 Biggest AI Stories of 2023 - Stanford HAI', 'href': 'https://hai.stanford.edu/news/13-biggest-ai-stories-2023', 'body': "2023 State of AI in 14 Charts. The sixth annual AI Index hit in early 202

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

[{'title': 'The Vital Role Of Partnerships In Scaling Artificial Intelligence - Forbes', 'href': 'https://www.forbes.com/sites/bernardmarr/2023/11/06/the-vital-role-of-partnerships-in-scaling-artificial-intelligence/', 'body': "To harness AI's $4.4 trillion potential, businesses are turning to strategic partnerships, filling skill and infrastructure gaps, and collaborating on best practices."}, {'title': 'The Keys To 2023: Strategic Partnerships, Scalable Integration And ...', 'href': 'https://www.forbes.com/councils/forbestechcouncil/2023/03/08/the-keys-to-2023-strategic-partnerships-scalable-integration-and-digital-transformation/', 'body': 'In 2023, companies will likely seek partnerships that help customers tap into emerging technologies, new markets and higher levels of expertise without requiring specialized knowledge or resources ...'}, {'title': '4 ways public-private partnerships can bridge the AI opportunity gap', 'href': 'https://www.weforum.org/stories/2024/01/public-pri

### Task Output ###
# Briefing Document: AI Advancements and Strategic Partnerships in 2023

## Participant Bios
- **[Participant Name 1]**: [Title, Company, Brief Bio]
- **[Participant Name 2]**: [Title, Company, Brief Bio]
- **[Participant Name 3]**: [Title, Company, Brief Bio]
- **[Participant Name 4]**: [Title, Company, Brief Bio]

## Industry Overview
### Current Landscape of AI Advancements
2023 has been a pivotal year for artificial intelligence, characterized by transformative advancements and 
widespread adoption. The generative AI sector has particularly flourished, with substantial funding of 
approximately **$25.2 billion**, showcasing robust industry confidence. Notable trends include:

- **Expansion of Generative AI**: This technology has moved beyond laboratory settings into practical applications,
influencing various sectors from creative industries to business operations.
- **Democratization of AI Tools**: There is a notable shift allowing smaller enterprises and individual innovators 
to access advanced AI capabilities, fostering the emergence of new markets and opportunities.

### Regulatory Environment
As AI technologies proliferate, so does regulatory scrutiny. Companies must navigate an evolving landscape of 
ethical considerations and privacy regulations to ensure compliance and protect user data.

### Collaborative AI Trends
The industry is witnessing a shift towards collaborative AI systems. These systems enhance productivity by 
improving human-AI interactions, particularly in critical sectors such as healthcare and cybersecurity.

### Partnership Opportunities
Public-private partnerships are emerging as crucial to addressing the AI opportunity gap, especially in workforce 
reskilling and upskilling initiatives. Examples such as the collaboration between **Microsoft and OpenAI** 
illustrate the potential for joint innovation to drive significant advancements in AI capabilities.

## Strategic Talking Points
1. **Innovation through Partnerships**: Strategic partnerships can lead to pioneering AI solutions that exceed the 
capabilities of individual organizations.
2. **Shared Vision for AI Development**: Establishing a common vision can align efforts to address industry trends 
and challenges, fostering collaboration.
3. **Long-term Strategic Alignment**: Ensuring both parties benefit from evolving AI initiatives is vital for 
sustained success.
4. **Leveraging Data and Insights**: Sharing data and insights can enhance AI development and improve 
decision-making in collaborative projects.
5. **Creating Value for Stakeholders**: Partnerships can generate value for organizations, stakeholders, customers,
and communities through responsible AI deployment.

## Strategic Recommendations
### Collaboration Opportunities
- Identify specific areas for collaboration to leverage advancements in AI for mutual benefit.
- Align research goals to foster innovation and value creation.

### Addressing Industry Challenges
- Develop strategies to tackle the talent shortage in AI, focusing on knowledge transfer and skills development 
through partnerships.
- Work collaboratively to navigate the regulatory landscape surrounding AI technologies.

### Enhancing Human-AI Collaboration
- Explore initiatives that promote the collaborative use of AI systems within respective sectors.
- Ensure ethical considerations are integrated into AI projects through comprehensive partnership frameworks.

### Exploring New Markets
- Target emerging markets with specific localized AI solutions to tap into new customer bases.
- Identify effective partnership models to accelerate AI adoption in these regions.

## Conclusion
This briefing serves as a foundation for discussions on potential collaborative projects in the tech industry, 
particularly in the realm of AI. By leveraging the insights and strategies outlined, participants can better 
navigate the challenges and opportunities presented by the rapidly evolving AI landscape.

None
